In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import re
import statsmodels.api as sm
import statsmodels.formula.api as smf
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import cross_validation
from datetime import datetime, timedelta
import calendar
import holidays
from dateutil.relativedelta import relativedelta

%matplotlib inline

In [34]:
#viz setup
# sns.set(style='whitegrid',font_scale=1.75,rc={"axes.spines.top":False,"axes.spines.right":False, "lines.linewidth": 2.5,'lines.markersize': 10},color_codes=False,palette=sns.color_palette(['#27a3aa','#f76d23','#70d6e3','#ffbb31','#b1c96d','#cce18a','#1c4c5d','#787642']))
sns.set(style='whitegrid',font_scale=1.5,rc={"axes.spines.top":False,"axes.spines.right":False, "lines.linewidth": 2.5,'lines.markersize': 10},color_codes=False,palette=sns.color_palette(['#27a3aa','#f76d23','#70d6e3','#ffbb31','#b1c96d','#cce18a','#1c4c5d','#787642']))

In [35]:
states = ["AL", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
regions = ["_ENC","_MAC","_MTN","_NEC","_PAC","PUS","_WNC","_WSC","_ESC","_SAC"]
sectors = ['RES','COM']

In [36]:
idx = 0
for state in states:
    for sector in sectors:
        response_consumption = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=ELEC.SALES.{}-{}.M".format(state,sector))
        j_consumption = response_consumption.json()
        tmp_consumption = pd.DataFrame(j_consumption['series'][0]['data'],columns=['month','sales_mkwh'])
        tmp_consumption['state'] = state
        tmp_consumption['sector'] = sector
        
        response_consumers = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=ELEC.CUSTOMERS.{}-{}.M".format(state,sector))
        j_consumers = response_consumers.json()
        tmp_consumers = pd.DataFrame(j_consumers['series'][0]['data'],columns=['month','consumers'])
        tmp_consumers['state'] = state
        tmp_consumers['sector'] = sector
        
        response_price = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=ELEC.PRICE.{}-{}.M".format(state,sector))
        j_price = response_price.json()
        tmp_price = pd.DataFrame(j_price['series'][0]['data'],columns=['month','price'])
        tmp_price['state'] = state
        tmp_price['sector'] = sector
        
        tmp = tmp_consumption.merge(tmp_consumers,how='left',on=['month','state','sector']).merge(tmp_price,how='left',on=['month','state','sector'])
        
        if idx == 0:
            energy_data = tmp.copy()
        else:
            energy_data = energy_data.append(tmp)
        idx = idx +1

In [37]:
idx = 0
for region in regions:
    response_cool = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=STEO.ZWCD{}.M".format(region))
    j_cool = response_cool.json()
    tmp_cool = pd.DataFrame(j_cool['series'][0]['data'],columns=['month','cooling_days'])
    tmp_cool['region'] = region
    
    response_heat = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=STEO.ZWHD{}.M".format(region))
    j_heat = response_heat.json()
    tmp_heat = pd.DataFrame(j_heat['series'][0]['data'],columns=['month','heating_days'])
    tmp_heat['region'] = region
    
    tmp = tmp_cool.merge(tmp_heat,how='left',on=['month','region'])
    if idx == 0:
        heating_cooling_days = tmp.copy()
    else:
        heating_cooling_days = heating_cooling_days.append(tmp)
    idx = idx +1

In [38]:
energy_data['revenue'] = energy_data.sales_mkwh*energy_data.price
country = energy_data.groupby(['month','sector']).sum().reset_index()
country['state'] = 'USA'
country.price = country.revenue/country.sales_mkwh

In [39]:
energy_data = energy_data.append(country)

In [40]:
energy_data['use_per_capita'] = energy_data.sales_mkwh*1000000/energy_data.consumers

In [41]:
heating_cooling_days.region = [re.sub('_','',r) for r in heating_cooling_days.region]

In [42]:
states.extend(['USA'])

In [43]:
state_region_mapping = pd.DataFrame(data={'state': states})

In [44]:
state_region_mapping['region'] = ''

In [45]:
state_region_mapping.loc[state_region_mapping.state.isin(['WA','OR','CA']),'region'] = 'PAC'
state_region_mapping.loc[state_region_mapping.state.isin(['MT','ID','WY','NV','UT','CO','AZ','NM']),'region'] = 'MTN'
state_region_mapping.loc[state_region_mapping.state.isin(['ND','SD','MN','NE','IA','KS','MO']),'region'] = 'WNC'
state_region_mapping.loc[state_region_mapping.state.isin(['OK','TX','AR','LA']),'region'] = 'WSC'
state_region_mapping.loc[state_region_mapping.state.isin(['WI','IL','IN','MI','OH']),'region'] = 'ENC'
state_region_mapping.loc[state_region_mapping.state.isin(['KY','TN','MS','AL']),'region'] = 'ESC'
state_region_mapping.loc[state_region_mapping.state.isin(['WV','MD','DE','VA','NC','SC','GA','FL','DC']),'region'] = 'SAC'
state_region_mapping.loc[state_region_mapping.state.isin(['NY','PA','NJ']),'region'] = 'MAC'
state_region_mapping.loc[state_region_mapping.state.isin(['RI','CT','MA','NH','VT','ME']),'region'] = 'NEC'
state_region_mapping.loc[state_region_mapping.state.isin(['USA']),'region'] = 'PUS'

In [46]:
energy_data = energy_data.merge(state_region_mapping,how='left',on='state')

In [47]:
energy_data= energy_data.merge(heating_cooling_days,how='left',on=['month','region'])

In [48]:
energy_data = energy_data.dropna()

In [49]:
energy_data = pd.concat([energy_data,pd.get_dummies(energy_data.sector)],axis=1)

In [50]:
energy_data['time'] = [12*(int(d[0:4])-2008)+int(d[4:6]) for d in energy_data.month]

In [51]:
energy_data['year'] = [int(d[0:4]) for d in energy_data.month]
energy_data['mon'] = [int(d[4:6]) for d in energy_data.month]

In [52]:
def get_season(m):
    if (m == 12)|(m<=2):
        return 'winter'
    if (m>=3)&(m<=5):
        return 'spring'
    if(m>=6)&(m<=8):
        return 'summer'
    if(m>=9)&(m<=11):
        return 'fall'

In [53]:
energy_data['season'] = energy_data.mon.apply(get_season)

In [54]:
energy_data.head()

,month,sales_mkwh,state,sector,consumers,price,revenue,use_per_capita,region,cooling_days,heating_days,COM,RES,time,year,mon,season
0,202101,3164.80449,AL,RES,2284207.0,12.38,39180.279586,1385.515625,ESC,5.622370,736.449637,0,1,157,2021,1,winter
1,202012,2992.76542,AL,RES,2326166.0,11.39,34087.598134,1286.565714,ESC,1.125024,726.404608,0,1,156,2020,12,winter
2,202011,2056.87325,AL,RES,2324212.0,12.99,26718.783518,884.976607,ESC,13.247228,343.815521,0,1,155,2020,11,fall
3,202010,2150.98005,AL,RES,2326554.0,13.17,28328.407259,924.534763,ESC,67.266534,155.426795,0,1,154,2020,10,fall
4,202009,2876.36566,AL,RES,2325509.0,13.18,37910.499399,1236.875738,ESC,210.462990,19.843894,0,1,153,2020,9,fall


In [55]:
energy_data['date'] = [datetime(y,m,1) for y,m in zip(energy_data.year, energy_data.mon)]

In [56]:
def get_datetime_features(date):
    st = date
    en = date + relativedelta(months=1) - relativedelta(days=1)
    
    ## number of days in month
    num_days = len(pd.date_range(st,en))
    ## number of weekends in month
    num_weekends = pd.date_range(st,en).weekday.isin([5,6]).sum()
    ## number of holidays in month
    us_holidays = holidays.US(years=date.year)
    us_holidays = pd.DataFrame(us_holidays.items(),columns=['date','hol'])
    us_holidays['date'] = pd.to_datetime(us_holidays.date)
    num_holidays = len(us_holidays[(us_holidays.date.dt.month == date.month) & ~(us_holidays.date.dt.weekday.isin([5,6]))])
    
    num_weekends_or_holidays = num_holidays+num_weekends
    ## % of weekdays in month
    pct_weekdays = 1 - (num_holidays+num_weekends)/num_days
    
    return num_days, num_weekends_or_holidays, pct_weekdays

In [57]:
energy_data['num_days'], energy_data['num_hols'], energy_data['pct_weekdays'] = zip(*energy_data.date.apply(get_datetime_features)) 

In [58]:
energy_data['y'] = energy_data.use_per_capita/energy_data.num_days

In [59]:
energy_data.head()

,month,sales_mkwh,state,sector,consumers,price,revenue,use_per_capita,region,cooling_days,...,RES,time,year,mon,season,date,num_days,num_hols,pct_weekdays,y
0,202101,3164.80449,AL,RES,2284207.0,12.38,39180.279586,1385.515625,ESC,5.622370,...,1,157,2021,1,winter,2021-01-01,31,12,0.612903,44.694052
1,202012,2992.76542,AL,RES,2326166.0,11.39,34087.598134,1286.565714,ESC,1.125024,...,1,156,2020,12,winter,2020-12-01,31,9,0.709677,41.502120
2,202011,2056.87325,AL,RES,2324212.0,12.99,26718.783518,884.976607,ESC,13.247228,...,1,155,2020,11,fall,2020-11-01,30,11,0.633333,29.499220
3,202010,2150.98005,AL,RES,2326554.0,13.17,28328.407259,924.534763,ESC,67.266534,...,1,154,2020,10,fall,2020-10-01,31,10,0.677419,29.823702
4,202009,2876.36566,AL,RES,2325509.0,13.18,37910.499399,1236.875738,ESC,210.462990,...,1,153,2020,9,fall,2020-09-01,30,9,0.700000,41.229191


In [60]:
energy_data.tail()

,month,sales_mkwh,state,sector,consumers,price,revenue,use_per_capita,region,cooling_days,...,RES,time,year,mon,season,date,num_days,num_hols,pct_weekdays,y
24095,202011,98994.84172,USA,RES,136421078.0,13.287650,1.315409e+06,725.656498,PUS,32.066798,...,1,155,2020,11,fall,2020-11-01,30,11,0.633333,24.188550
24096,202012,103951.57447,USA,COM,18796159.0,10.419035,1.083075e+06,5530.468989,PUS,7.056622,...,0,156,2020,12,winter,2020-12-01,31,9,0.709677,178.402225
24097,202012,129037.22650,USA,RES,137531520.0,12.756682,1.646087e+06,938.237478,PUS,7.056622,...,1,156,2020,12,winter,2020-12-01,31,9,0.709677,30.265725
24098,202101,103755.63610,USA,COM,18733383.0,10.251220,1.063622e+06,5538.542403,PUS,9.984711,...,0,157,2021,1,winter,2021-01-01,31,12,0.612903,178.662658
24099,202101,136791.50557,USA,RES,136154434.0,12.643004,1.729456e+06,1004.679037,PUS,9.984711,...,1,157,2021,1,winter,2021-01-01,31,12,0.612903,32.409001


In [61]:
energy_data.date.max()

Timestamp('2021-01-01 00:00:00')

In [31]:
energy_data.to_csv('energy_data.csv',index=False)